In [6]:
import goodfire
client = goodfire.Client(api_key="sk-goodfire-a5ADTw9c-ONSj-vU8G78pKW3TwnmuLK_j0guBDkMWsGDpA_MDH2gFg")

variant = goodfire.Variant("meta-llama/Meta-Llama-3.1-8B-Instruct")


# edits = client.features.AutoSteer(
#     # specification="Talk with a lot of puns",
#     model = variant
# )
print(variant)



Variant(
   base_model=meta-llama/Meta-Llama-3.1-8B-Instruct,
   edits={
   }
   scopes={
   }
)


In [20]:
output = client.chat.completions.create(
    [{"role": "user", "content": "Tell me about godzilla"}],
    model = variant,
    stream = True,
    max_completion_tokens=120
)

In [53]:
act = client.features.activations([{"role": "user", "content": message_text}], model=variant)

In [21]:
chunks = list(output)

In [50]:
role, message_text = extract_message(chunks)
type(message_text)

str

In [25]:
full_text = []
for ch in chunks:
    delta = ch.choices[0].delta.content or ""
    print(delta,end="")
    full_text.append(delta)
text = "".join(full_text)

Godzilla! The King of the Monsters. He's a giant, fire-breathing, city-stomping reptile from Japanese legend. Created by Tomoyuki Tanaka in 1954, Godzilla was meant to be a metaphor for the atomic age and the fears of nuclear destruction.

Godzilla's real name is Gojira, and he's often referred to as "King Kong's nemesis." He's typically around 100-150 meters tall, with incredible strength and speed. He's also highly resistant to radiation and can even withstand nuclear explosions.

Over 65 years, there have

In [31]:
for token in chunks:
    print(token.choices[0])

index=0 delta=StreamingDelta(role='assistant', content='God') gf_token_index=68 finish_reason=None
index=0 delta=StreamingDelta(role='assistant', content='zilla') gf_token_index=69 finish_reason=None
index=0 delta=StreamingDelta(role='assistant', content='!') gf_token_index=70 finish_reason=None
index=0 delta=StreamingDelta(role='assistant', content=' The') gf_token_index=71 finish_reason=None
index=0 delta=StreamingDelta(role='assistant', content=' King') gf_token_index=72 finish_reason=None
index=0 delta=StreamingDelta(role='assistant', content=' of') gf_token_index=73 finish_reason=None
index=0 delta=StreamingDelta(role='assistant', content=' the') gf_token_index=74 finish_reason=None
index=0 delta=StreamingDelta(role='assistant', content=' Monsters') gf_token_index=75 finish_reason=None
index=0 delta=StreamingDelta(role='assistant', content='.') gf_token_index=76 finish_reason=None
index=0 delta=StreamingDelta(role='assistant', content=' He') gf_token_index=77 finish_reason=None
in

In [54]:
features = client.features.search(
    "Godzilla",
    model = variant,
    top_k=5
)
features
cols = [f.index_in_sae for f in features]

feat_act = act[:,cols]
feat_act

array([[0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.

In [48]:
def extract_message(chunks, choice_idx=0):
    role = "assistant"
    parts = []
    for ch in chunks:
        delta = ch.choices[choice_idx].delta
        if getattr(delta, "role", None):
            role = delta.role
        if getattr(delta, "content", None):
            parts.append(delta.content)
    text = "".join(parts)
    return role, text